In [1]:
pip install gspread oauth2client smtplib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib


In [11]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Google Sheets API 인증 설정
def authenticate_google_sheets(credentials_file):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(credentials)
    return client


# Google Sheets 데이터 읽기
def load_sheet_data(client, spreadsheet_id, sheet_name):
    sheet = client.open_by_key(spreadsheet_id).worksheet(sheet_name)
    data = pd.DataFrame(sheet.get_all_records())
    return data

# SP담당자별 데이터 필터링
def filter_data_by_sp(data, sp_manager_column, sp_manager):
    return data[data[sp_manager_column].str.lower() == sp_manager.lower()]

# HTML 테이블 생성
def create_html_table(dataframe):
    if dataframe.empty:
        return "<p>데이터가 없습니다.</p>"
    return dataframe.to_html(index=False, border=1, classes="table", justify="left")

# 이메일 전송 함수
def send_email(smtp_server, port, login, password, recipient, subject, body, cc=None):
    msg = MIMEMultipart()
    msg["From"] = login
    msg["To"] = recipient
    msg["Subject"] = subject
    if cc:
        msg["Cc"] = cc
    msg.attach(MIMEText(body, "html"))

    try:
        with smtplib.SMTP(smtp_server, port) as server:
            server.starttls()
            server.login(login, password)
            server.sendmail(login, [recipient] + ([cc] if cc else []), msg.as_string())
        print(f"이메일 발송 완료: {recipient}")
    except Exception as e:
        print(f"이메일 발송 오류: {e}")


# 메인 함수
def main():
    # 인증 및 Google Sheets 설정
    credentials_file = "/content/drive/MyDrive/Key/credentials.json"  # JSON 키 파일 경로
    spreadsheet_id = "1rC2VatA-QczdN5mP9MXqWXCNREfAiUkfUu2XtkJ6sT0"  # Google 스프레드시트 ID
    db_sheet_name = "db"  # DB 시트 이름
    filter_sheet_name = "장기설변"  # 장기설변 시트 이름
    mail_sheet_name = "메일관리"  # 메일관리 시트 이름

    client = authenticate_google_sheets(credentials_file)

    # 시트 데이터 로드
    db_data = load_sheet_data(client, spreadsheet_id, db_sheet_name)
    filter_data = load_sheet_data(client, spreadsheet_id, filter_sheet_name)
    mail_data = load_sheet_data(client, spreadsheet_id, mail_sheet_name)

    # 메일 설정
    smtp_server = "smtp.gmail.com"
    port = 587
    email_account = "bough38@naver.com"  # 발신 이메일 계정
    email_password = "see0303!!"  # 발신 이메일 계정 비밀번호

    sp_manager_column = "SP담당자명"  # SP담당자명 컬럼 이름
    email_column = "이메일"
    cc_column = "참조자메일"

    # 메일관리 시트의 각 SP담당자에 대해 작업
    for _, mail_row in mail_data.iterrows():
        sp_manager = mail_row.get(sp_manager_column)
        recipient_email = mail_row.get(email_column)
        cc_email = mail_row.get(cc_column)

        if not sp_manager or not recipient_email:
            print(f"SP담당자명 또는 이메일이 없습니다: {mail_row}")
            continue

        # SP담당자명별 데이터 필터링
        filtered_db_data = filter_data_by_sp(db_data, sp_manager_column, sp_manager)
        filtered_filter_data = filter_data_by_sp(filter_data, sp_manager_column, sp_manager)

        # 데이터가 없는 경우 이메일 전송 생략
        if filtered_db_data.empty and filtered_filter_data.empty:
            print(f"{sp_manager}님의 데이터가 없습니다. 메일을 전송하지 않습니다.")
            continue

        # HTML 테이블 생성
        db_table = create_html_table(filtered_db_data)
        filter_table = create_html_table(filtered_filter_data)

        # 이메일 본문 작성
        html_body = f"""
        <p>안녕하세요,</p>
        <p>{sp_manager}님의 데이터는 아래와 같습니다:</p>
        <h3>DB 데이터</h3>
        {db_table}
        <h3>장기설변 데이터</h3>
        {filter_table}
        <p>감사합니다.</p>
        """

        # 이메일 전송
        send_email(
            smtp_server,
            port,
            email_account,
            email_password,
            recipient_email,
            f"{sp_manager}님의 데이터 공유",
            html_body,
            cc=cc_email
        )

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Key/credentials.json'